# CDC Acceptance Test - Ardumoto

### INTRODUCTION:

This test is designed as part of acceptance testing of CDC produced Ardumoto  
[Ardumoto](https://www.sparkfun.com/products/14180).
Ardumoto can supports upto two DC motor, the test drives both the motors at different speed and direction to ensure module's integrity.

### BACKGROUND:

The Ardumoto shield can be mounted on the PYNQ Board similar to PYNQ arduino shield:
Motor A and Motor B are connected as below to the arduino pins:

| Pin number | Functionality                    |
|------------|----------------------------------|
| 2          | Direction control for motor A    |
| 3          | PWM control (speed) for motor A  |
| 4          | Direction control for motor B    |
| 11         | PWM control (speed) for motor B  |

### Motor Connections

![alt text](ardumoto_wiring_cdc.jpg)


## Setup the Motor Driver Validation Test

In [1]:
# Setup base PYNQ environment
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# Import sleep
from time import sleep

# Define constants to control motor direction
MOTOR_A = 0
MOTOR_B = 1
POLAR_DEFAULT = 0
POLAR_REVERSE = 1
CLOCKWISE = 0
ANTICLOCKWISE = 1

In [2]:
%%microblaze base.ARDUINO

#include "xio_switch.h"
#include "gpio.h"
#include "timer.h"

#define DEFAULT_PERIOD 625998
#define DEFAULT_DUTY 312998

#define PWM_A_PIN 3
#define PWM_B_PIN 11
#define DIR_A_PIN 2
#define DIR_B_PIN 4

typedef enum motor {
MOTOR_A = 0,
MOTOR_B = 1,
}motor_e;

static unsigned int pol_a = 0, pol_b = 0;
static unsigned int dir_a = 0, dir_b = 0;
static unsigned int duty_a = 50, duty_b = 50;

static timer timer_a;
static timer timer_b;
static gpio gpio_a;
static gpio gpio_b;

unsigned int init_ardumoto(){    
    timer_a = timer_open_device(0);
    timer_b = timer_open_device(5);
    set_pin(PWM_A_PIN, PWM0);
    set_pin(PWM_B_PIN, PWM5);
    gpio_a = gpio_open(DIR_A_PIN);
    gpio_b = gpio_open(DIR_B_PIN);
    gpio_set_direction(gpio_a, GPIO_OUT);
    gpio_set_direction(gpio_b, GPIO_OUT);
    return 0;
}

void configure_polar(unsigned int motor, unsigned int polarity){
    if (motor == MOTOR_A) {
        pol_a = polarity;
    }else if (motor == MOTOR_B) {
        pol_b = polarity;
    }
}

void set_direction(unsigned int motor, unsigned int direction){
    if (motor == MOTOR_A){
        dir_a = (direction)? pol_a : !pol_a;
    }
    else if (motor == MOTOR_B){
        dir_b = (direction)? pol_b : !pol_b;
    }
}

void set_speed(unsigned int motor, unsigned int speed){
    if (motor == MOTOR_A) {
        duty_a = speed;
    } else if (motor == MOTOR_B) {
        duty_b = speed;
    }
}

void run(unsigned int motor){
    if (motor == MOTOR_A) {
        gpio_write(gpio_a, dir_a);
        timer_pwm_generate(timer_a, DEFAULT_PERIOD, 
                           duty_a*DEFAULT_PERIOD/100);
    }else if(motor == MOTOR_B) {
        gpio_write(gpio_b, dir_b);
        timer_pwm_generate(timer_b, DEFAULT_PERIOD, 
                           duty_b*DEFAULT_PERIOD/100);
    }
}

void stop(unsigned int motor){
    if (motor == MOTOR_A) {
        timer_pwm_stop(timer_a);
    }else if (motor == MOTOR_B){
        timer_pwm_stop(timer_b);
    }
}

In [3]:
init_ardumoto()

configure_polar(MOTOR_A, POLAR_DEFAULT) 
configure_polar(MOTOR_B, POLAR_DEFAULT)

# CDC Test 1: Run Both Motors Clockwise at Low Speed

### Test Aim: 
Check if the Ardumoto shield can run both motors clockwise at low speed

### Test Expectation: 
Both motors run clockwise at the same low speed (10%)

In [4]:
from time import sleep

set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 8)
set_speed(MOTOR_B, 8)

print('Running Motors A and B at 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 10% speed for 3 seconds
Stopping Motors A and B


# CDC Test 2: Run Both Motors Counterclockwise at Low Speed

### Test Aim: 
Check if the Ardumoto shield can run both motors counterclockwise at low speed

### Test Expectation: 
Both motors run counterclockwise at the same low speed (10%)

In [5]:
from time import sleep

set_direction(MOTOR_A, ANTICLOCKWISE)
set_direction(MOTOR_B, ANTICLOCKWISE)
set_speed(MOTOR_A, 10)
set_speed(MOTOR_B, 10)

print('Running Motors A and B at 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 10% speed for 3 seconds
Stopping Motors A and B


# CDC Test 3: Run Both Motors in Opposite Directions at Low Speed

### Test Aim: 
Check if the Ardumoto Shield can run both motors at low speed in opposite directions

### Test Expectation: 
Both motors run in opposite directions at the same low speed (10%)

In [6]:
from time import sleep

set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, ANTICLOCKWISE)
set_speed(MOTOR_A, 10)
set_speed(MOTOR_B, 10)

print('Running Motors A and B and 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B and 10% speed for 3 seconds
Stopping Motors A and B


# CDC Test 4: Run Both Motors Clockwise at High Speed

### Test Aim: 
Check if the Ardumoto shield can run both motors clockwise at high speed

### Test Expectation: 
Both motors run clockwise at the same high speed (90%)

In [7]:
from time import sleep

set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 90)
set_speed(MOTOR_B, 90)

print('Running Motors A and B at 90% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 90% speed for 3 seconds
Stopping Motors A and B


# CDC Test 5: Run Both Motors at Counterclockwise at High Speed

### Test Aim: 
Check if the Ardumoto Shield can run both motors counterclockwise at high speed

### Test Expectation: 
Both motors run counterclockwise at the same high speed (90%)

In [8]:
from time import sleep

set_direction(MOTOR_A, ANTICLOCKWISE)
set_direction(MOTOR_B, ANTICLOCKWISE)
set_speed(MOTOR_A, 90)
set_speed(MOTOR_B, 90)

print('Running Motors A and B and 90% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B and 90% speed for 3 seconds
Stopping Motors A and B


# CDC Test 6: Run Both Motors in Opposite Directions at High Speed

### Test Aim: 
Check if the Ardumoto Shield can run both motors at high speed in opposite directions

### Test Expectation: 
Both motors run in opposite directions at the same high speed (90%)

In [9]:
from time import sleep

set_direction(MOTOR_A, ANTICLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 90)
set_speed(MOTOR_B, 90)

print('Running Motors A and B at 90% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 90% speed for 3 seconds
Stopping Motors A and B
